# General statistics on the dataset

Shows table 1 in the paper

In [ ]:
import sys
import ast
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset, concatenate_datasets, disable_caching

In [ ]:
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
disable_caching()
pd.options.mode.chained_assignment = None
pd.options.display.max_colwidth = 100

In [ ]:
from src.common import dist_funcs, new_assignments_id

In [ ]:
start_df = concatenate_datasets(list(load_dataset("koutch/intro_prog", "dublin_data").values())).to_pandas()
start_df = start_df.replace(new_assignments_id)
start_df

In [ ]:
df = concatenate_datasets(list(load_dataset("koutch/intro_prog", "dublin_repair", 
                                            download_mode="force_redownload",
                                            ignore_verifications=True).values())).to_pandas()
df = df.replace(new_assignments_id)
df

In [ ]:
grouped_by = df.groupby("assignment_id")

In [ ]:
# number of annotations per assignment
n_submissions = grouped_by.count()["submission_id"]
n_submissions.name = "#SS"
# number of students 
n_students = grouped_by.apply(lambda g: len(g.user.unique()))
n_students.name = "#STU"
n_students
# number of non-empty annotations
n_non_empty_annot = grouped_by.apply(lambda g: g.annotation.astype(bool).sum())
n_non_empty_annot.name = "#AN"

def count_avg_lines(code):
    return len(code.splitlines())

# average number of lines per submission
avg_lines = grouped_by.apply(lambda g: g[g.annotation.astype(bool)].annotation.apply(count_avg_lines).mean())
avg_lines = avg_lines.round(2)
avg_lines.name = "#lines"
avg_lines

In [ ]:
ori_corr_submissions = start_df.groupby("assignment_id").apply(lambda gdf: gdf[gdf.correct].count())["submission_id"][n_students.index]
ori_corr_submissions.name = "#CC"
ori_corr_submissions, ori_corr_submissions.sum()

In [ ]:
ori_buggy_submissions = start_df.groupby("assignment_id").apply(lambda gdf: gdf[~gdf.correct].count())["submission_id"][n_students.index]
ori_buggy_submissions.name = "#BC"
ori_buggy_submissions, ori_buggy_submissions.sum()

In [ ]:
descriptions = start_df.groupby("assignment_id").description.apply(lambda gdf: gdf.iloc[0])
descriptions = descriptions[n_students.index]
descriptions

In [ ]:
statistics = pd.concat([descriptions, ori_corr_submissions, ori_buggy_submissions,
                        n_non_empty_annot, avg_lines, n_students], axis=1)
statistics.index.name = ""
statistics

### The final table

In [ ]:
statistics.loc["total/average"] = statistics.sum(axis=0)
statistics.loc["total/average", "description"] = "" # removing the description for the last one 
statistics.loc["total/average", "#lines"] = statistics["#lines"].mean()
statistics["#lines"] = statistics["#lines"].round(2)
statistics

In [ ]:
# print(statistics.to_latex(multicolumn=True, multirow=True, column_format='rlccccc'))